# Python / UNIX code used in ACAD2018 lecture.

Below are the example Python code I use, so you can easily copy and paste the following into Terminal and/or IPython. 

Note that I added our ```scripts/``` directory to the PATH and PYTHONPATH. This lets us use the R script and Python module from any folder.

First, note that many files are in the ```lecture/``` folder. Let's move them out of this folder into a new folder we make, called ```oldfiles/```. We'll be making all of these files throughout the day, but in case you have problems, at least you have these for reference. 

## Lecture 1: f3-statistics

### 1.1. convertf - Making the PAR file. 
Type ```ipython``` into the Terminal and then the following code, changing your directory as needed. Check your outpd folder for the PAR file. To check out what the python function I wrote does, type ```ff.parfile?``` into IPython after ```import fstat_funcs as ff```.

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  ##Directory your data (IND/SNP/GENO) are in.
outpd=home+"/workshop/3_Wed_Yang/lecture/"  ##Directory you want to put PAR file in.
fh="data"
pt="CONVERTF"
label="EIG"
convertf="EIGENSTRAT"
ff.parfile(pD,fh,pt,label,convertf=convertf,outpd=outpd)

Now in Terminal, in your ```outpd``` folder, type the following to convert the data from ANCESTRYMAP format to EIGENSTRAT format. It should take about two minutes. 
```
convertf -p data.CONVERTF.EIG.par > data.CONVERTF.EIG.log
```

Now, we should be able to use ```less``` to look at the new GENO file.

### 1.2. qp3Pop - Making the PAR and POP file. 

In IPython, to make the PAR file, type the following. Note that I'm using the same function as above. 

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
pt="f3"
label="outf3_MA1"

ff.parfile(pD,fh,pt,label,outpd=outpd)

To make the POP file, we can use the following:

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
label="outf3_MA1"
out=["Mbuti"]
s1=["MA1"]
ff.f3pop(pD,fh,label,Target=out,S1=s1,outpd=outpd)

### 1.3 Running qp3Pop

Now in Terminal, type the following to calculate f3-statistics for Loschbour.
```
qp3Pop -p data.EIG.f3.outf3_MA1.par > data.EIG.f3.outf3_MA1.log
```

And to view the sorted results:
```
cat data.EIG.f3.outf3_MA1.log | grep result | awk '{print $2,$3,$4,$5,$6}' | sort -rk4,4 > sorteddata.EIG.f3.outf3_MA1.log 

```

### 1.4 Visualizing qp3Pop results in a heatmap

Next, I want to show you how to visualize a heatmap using a mixture of Python and R. For this, we will use a readied f3 log file:
```~/workshop/3_Wed_Yang/lecture/data.EIG.f3.outf3_ALL.log```
where I ran  all combinations as Source1 and Source2, in f3(Source1, Source2; Mbuti). 

Using the script below, I specify a subset (```anceur```), make an array of f3 values, and then use this array to make a CSV file of the array. 

In [ ]:
import fstats_funcs as ff
import pandas as pandas
import os

home=os.environ['HOME'] 

pd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
label="outf3_ALL"
anceur=["HungaryGamba_BA","HungaryGamba_CA","HungaryGamba_IA","HungaryGamba_HG",
        "HungaryGamba_EN","SwedenSkoglund_MHG","SwedenSkoglund_NHG","SwedenSkoglund_MN",
        "Iceman","Motala_HG","LBK_EN","Unetice_EBA","Corded_Ware_LN",
        "Bell_Beaker_LN","Karelia_HG","Halberstadt_LBA",
        "Alberstedt_LN","Samara_HG","Esperstedt_MN","Starcevo_EN","LBKT_EN","Yamnaya",
        "Spain_MN","Spain_EN","Karsdorf_LN","Baalberge_MN","Stuttgart",
        "Loschbour","LaBrana1","Kostenki14"]
others=["Ust_Ishim","MA1","English","Han","Dai","Karitiana","Mixe"]
mylst=anceur+others
pops=[mylst,mylst,"Mbuti"]
f3ary=ff.f3mkary(pd,"%s.f3.%s.log" % (fh,label),pops,forZ="f")
f3dat=pandas.DataFrame(f3ary)
rownames={ind:mylst[ind] for ind in range(len(mylst))}
f3dat=f3dat.rename(rownames, axis='columns')
f3dat=f3dat.rename(rownames, axis='index')
f3dat.to_csv(pd+"%s.f3.%s.ANC1.csv" % (fh,label), sep='\t')

In the ```scripts/``` folder, there should be an R script (```mkheatmap.R```) that looks like what is below. This is just a simple script to run the heatmap.2 function on our data and output a PDF with the resulting heatmap. 

```
#!/usr/bin/env Rscript
args = commandArgs(trailingOnly=TRUE)

library(gplots)
name=args[1]

myfile=read.csv(paste(name,".csv",sep=""),sep="\t",header=T)

mydat=subset(myfile, select = -c(X) )
rownames(mydat)=colnames(mydat)
pdf(paste(name,".pdf",sep=""))
#mydat[is.na(mydat)] <- 0
mydat=data.matrix(mydat)
v=heatmap.2(mydat,symkey=FALSE, trace="none",margins =c(10,10)) #,dendrogram="none")
x=rownames(mydat)[v$rowInd]
mynames=data.frame(x)
write.table(mynames,paste(name,".sorted.txt",sep=""),sep="\t",row.names=F,quote=F)
dev.off()
```

Then, run 
```
mkheatmap.R data.EIG.f3.outf3_ALL.ANC1
```

This should generate a PDF file with the heatmap of included individuals.

## Lecture 2: D-statistics

### 2.1 Making the PAR file
For the PAR file, note again I'm using the same function as for convertf and f3.

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
pt="D"
label="Set1_AllD_Mbuti"
ff.parfile(pD,fh,pt,label,outpd=outpd)

### 2.2 D-statistics, making the POP files

The list of populations under consideration include:
```
mypops=["Ust_Ishim","Kostenki14","MA1","Loschbour","Stuttgart",
        "Spain_MN","Yamnaya","Corded_Ware_LN","English",
        "Han","Dai","Karitiana","Mixe"]
```
These represent a wide array of Eurasians, especially ancient Europeans. If we run D-statistics placing them all in each position of P1, P2, and P3, the D-statistics we want to compare will be calculated (along with some other combinations). 

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
pt="D"
label="Set1_AllD_Mbuti"
ff.parfile(pD,fh,pt,label,outpd=outpd)
fh="data.EIG"
pt="D"
outs=["Mbuti"]
mypops=["Ust_Ishim","Kostenki14","MA1","Loschbour","Stuttgart",
        "Spain_MN","Yamnaya","Corded_Ware_LN","English",
        "Han","Dai","Karitiana","Mixe"]
label="Set1_AllD_Mbuti"
ff.Df4pop(pD,fh,pt,label,mypops,mypops,mypops,outs,outpd=outpd)

### 2.3 Running qpDstat

Now in Terminal, type the following to calculate D-statistics for the PAR and POP files designated above.
```
qpDstat -p data.EIG.D.Set1_AllD_Mbuti.par > data.EIG.D.Set1_AllD_Mbuti.log
```

This should take about three minutes.

### 2.4 Viewing results for qpDstat

In UNIX, we can pull out the specific D-statistic combinations we want to look at:

```
cat data.EIG.D.Set1_AllD_Mbuti.log | grep result | awk '{print $2,$3,$4,$5,$6,$7}' | awk '$1=="MA1" && $3=="Ust_Ishim"'

cat data.EIG.D.Set1_AllD_Mbuti.log | grep result | awk '{print $2,$3,$4,$5,$6,$7}' | awk '$3=="MA1" && $1=="Ust_Ishim"'

cat data.EIG.D.Set1_AllD_Mbuti.log | grep result | awk '{print $2,$3,$4,$5,$6,$7}' | awk '$1=="MA1" && $2=="Ust_Ishim"'

```

Another way of summarizing more data is to put them into a table. Usually, I compare one individual/population to a panel, using D(Panel, X; Panel, Mbuti) and D(Panel, Panel; X, Mbuti). With this particular construction, negative values along the column in both tables indicates that member of the panel forms a clade with X. I've also written two functions, ```ff.Dmkary()``` and ```ff.Dmat2xlsx()```, to easily convert the LOG file into a table in an XLSX file. 

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/lecture/"  
fh="data.EIG"
label="Set1_AllD_Mbuti"

mypops=["Ust_Ishim","Kostenki14","MA1","Loschbour","Stuttgart",
        "Spain_MN","Yamnaya","Corded_Ware_LN","English",
        "Han","Dai","Karitiana","Mixe"]

mainpop="MA1"
myary=ff.Dmkary(pD,fh+".D."+label+".log",[mypops,mypops,mainpop,"Mbuti"],"Z")
ff.Dmat2xlsx(myary,pD+fh+".P3_%s" % mainpop,"D(X,Y;%s,Mbuti)" % mainpop,mypops,mypops)

myary=ff.Dmkary(pD,fh+".D."+label+".log",[mypops,mainpop,mypops,"Mbuti"],"Z")
ff.Dmat2xlsx(myary,pD+fh+".P2_%s" % mainpop,"D(X,%s;Y,Mbuti)" % mainpop,mypops,mypops)

## Lecture 3: Admixture f3-statistics

We've already learned how to calculate f3-statistics, but we focused on calculating outgroup f3-statistics. Below, we create a PAR and POP file, but for admixture f3-statistics. 

### 3.1 Testing for gene flow into Native Americans
To test gene flow into Native Americans, we test f3(AMER; MA1, EAS). 

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
pt="f3"
label="EAS_MA1_AMER"
amer=["Karitiana","Mixe"]
eur=["Kostenki14","Loschbour","Stuttgart","English"]
eas=["Han","Dai"]
ff.parfile(pD,fh,pt,label,outpd=outpd,inbreed=True)
ff.f3pop(pD,fh,label,Target=amer,S1=eas,S2=["MA1"]+eur,outpd=outpd)

Then, to run ```qp3Pop```,
```
qp3Pop -p data.EIG.f3.EAS_MA1_AMER.par > data.EIG.f3.EAS_MA1_AMER.log
```

We are interested in negative results, so we can use:
```
cat data.EIG.f3.EAS_MA1_AMER.log | grep result | awk ' $5 < 0 '
```
What happened?

### 3.2 Testing for gene flow in the Corded_Ware_LN

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
pt="f3"
label="MN_Steppe_LN"

mn=["Spain_MN","Spain_EN","Baalberge_MN","Esperstedt_MN"]
steppe=["Yamnaya"]
ln=["Corded_Ware_LN","Unetice_EBA","Bell_Beaker_LN","Halberstadt_LBA"]
ff.parfile(pD,fh,pt,label,outpd=outpd,inbreed=True)
ff.f3pop(pD,fh,label,Target=ln,S1=mn,S2=steppe,outpd=outpd)

## Lecture 4: qpWave/qpAdm

For both qpWave and qpAdm, in addition to the PAR file, we require two POP files - one designating a set of outgroups ('rightpop') and the other designating the set of populations being tested for streams of ancestry and/or gene flow ('leftpop'). I wrote a simple function ```ff.mklst()``` to do this in Python. First, I make a list containing the set of outgroups I plan to use.

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/lecture/"

outgroups=["Ust_Ishim","Mbuti","Ju_hoan_North","Papuan","Kostenki14"]
fh="data.EIG"
ff.mklst(pD,fh,"out0",outgroups)

### 4.1 qpWave

Below, we make the PAR file and LEFTPOP list to use in qpWave. Note that no matter the order of the leftpop list, the result should be the same. 

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
pt="wave"
label="S_AMER_EAS_MA1"
out="out0"
ff.parfile(pD,fh,pt,label,outpd=outpd,outgroup=fh+"."+out)

ff.mklst(outpd,fh,pt+"."+label,["Karitiana","Han","MA1"],suffix="leftpop")

To run qpWave, in Unix we run the command:

```
qpWave -p data.EIG.wave.S_AMER_EAS_MA1.par > data.EIG.wave.S_AMER_EAS_MA1.log
```

This should take about 2.5 minutes. 
Use the following command to find the results we need to determine the minimum number of ancestral sources. 
``` 
grep f4rank data.EIG.wave.S_AMER_EAS_MA1.log
```

### 4.2 qpAdm
Next, we use qpAdm to determine who is admixed, and what the admixture proportions are. We make the PAR file similarly to above, but for the LEFTPOP, the ID in the first row will be the proposed target, with the subsequent rows listing the potential sources. We run all three combinations of qpAdm(Target; S1, S2) to determine which admixture models are possible.

In [ ]:
import fstats_funcs as ff
import os

home=os.environ['HOME']
pD=home+"/workshop/3_Wed_Yang/data/"  
outpd=home+"/workshop/3_Wed_Yang/lecture/"
fh="data.EIG"
pt="adm"
label="S_EAS_AMER_MA1"
out="out0"
ff.parfile(pD,fh,pt,label+"_1",outpd=outpd,outgroup=fh+"."+out)
ff.mklst(outpd,fh,pt+"."+label+"_1",["Han","Karitiana","MA1"],suffix="leftpop")

ff.parfile(pD,fh,pt,label+"_2",outpd=outpd,outgroup=fh+"."+out)
ff.mklst(outpd,fh,pt+"."+label+"_2",["MA1","Han","Karitiana"],suffix="leftpop")

ff.parfile(pD,fh,pt,label+"_3",outpd=outpd,outgroup=fh+"."+out)
ff.mklst(outpd,fh,pt+"."+label+"_3",["Karitiana","MA1","Han"],suffix="leftpop")

Then, we run ```qpAdm```, as shown below.
```
qpAdm -p data.EIG.adm.S_EAS_AMER_MA1_1.par > data.EIG.adm.S_EAS_AMER_MA1_1.log &
qpAdm -p data.EIG.adm.S_EAS_AMER_MA1_2.par > data.EIG.adm.S_EAS_AMER_MA1_2.log &
qpAdm -p data.EIG.adm.S_EAS_AMER_MA1_3.par > data.EIG.adm.S_EAS_AMER_MA1_3.log &
```